# Neural Network Project - Part 2 - summer 2024
### Mohammad Hossein Najafi - 97103938
---

### Convolution layer

In [23]:
import numpy as np

image = np.array([
    [0.51, 0.90, 0.88, 0.84, 0.05],
    [0.40, 0.62, 0.22, 0.59, 0.10],
    [0.11, 0.20, 0.74, 0.03, 0.14],
    [0.47, 0.00, 0.85, 0.70, 0.09],
    [0.76, 0.19, 0.72, 0.17, 0.57]
])

kernel = np.array([
    [-0.13, 0.15],
    [-0.51, 0.62]
])

# Convolution
def convolution(image, kernel):

    output_height = image.shape[0] - kernel.shape[0] + 1
    output_width = image.shape[1] - kernel.shape[1] + 1
    output = np.zeros((output_height, output_width))

    for i in range(output_height):
        for j in range(output_width):
            output[i, j] = np.sum(image[i:i + kernel.shape[0], j:j + kernel.shape[1]] * kernel)

    return output

# Perform the convolution
convolved = convolution(image, kernel)
print("Convolved Output:")
print(convolved)


Convolved Output:
[[ 0.2491 -0.1648  0.2652 -0.3406]
 [ 0.1089  0.3092 -0.2989  0.0098]
 [-0.224   0.612  -0.0912 -0.2841]
 [-0.3309  0.477  -0.2673  0.1892]]


### Max-pooling layer

In [24]:
# max-pooling
def max_pooling(input, pool_size):
    output_shape = (
        input.shape[0] // pool_size[0],
        input.shape[1] // pool_size[1]
    )
    output = np.zeros(output_shape)
    for y in range(0, input.shape[0], pool_size[0]):
        for x in range(0, input.shape[1], pool_size[1]):
            output[y // pool_size[0], x // pool_size[1]] = np.max(input[y:y + pool_size[0], x:x + pool_size[1]])
    return output

pooled = max_pooling(convolved, (2, 2))
print("Max-Pooled Output:")
print(pooled)

Max-Pooled Output:
[[0.3092 0.2652]
 [0.612  0.1892]]


### Sigmoid layer

In [25]:
# Sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

sigmoid_output = sigmoid(pooled)
print("\nOutput after applying sigmoid:")
print(sigmoid_output)


Output after applying sigmoid:
[[0.57668998 0.56591413]
 [0.64839689 0.5471594 ]]


### Fully Connected layer

In [26]:
# Fully Connected
fc_weights = np.array([
    [0.61, 0.82, 0.96, -1],
    [0.02, -0.5, 0.23, 0.17]
])
fc_biases = np.array([0, 0])  # Assuming biases are zero for simplicity
flattened = sigmoid_output.flatten()

fc_output = np.dot(flattened, fc_weights.T) + fc_biases
print("\nOutput after fully connected layer:")
print(fc_output)


Output after fully connected layer:
[ 0.89113209 -0.02927488]


### Backpropagation

In [36]:
# Backpropagation
delta_fc = [0.25,-0.15]

# Backpropagation fully connected layer
delta_flattened = np.dot(delta_fc, fc_weights)
delta_fc_weights = np.outer(delta_fc, flattened)
delta_fc_biases = delta_fc

print("fully connected delta input:")
print(delta_flattened)
print()
print("fully connected delta weights:")
print(delta_fc_weights)
print()
print("fully connected delta biases:")
print(delta_fc_biases)

fully connected delta input:
[ 0.1495  0.28    0.2055 -0.2755]

fully connected delta weights:
[[ 0.14417249  0.14147853  0.16209922  0.13678985]
 [-0.0865035  -0.08488712 -0.09725953 -0.08207391]]

fully connected delta biases:
[0.25, -0.15]


In [31]:
# Backpropagation sigmoid layer
sigmoid_derivative = sigmoid_output * (1 - sigmoid_output)
delta_sigmoid = delta_flattened.reshape(sigmoid_output.shape) * sigmoid_derivative

print("sigmoid delta:")
print(delta_sigmoid)

sigmoid delta:
[[ 0.03649574  0.06878349]
 [ 0.04684955 -0.06826229]]


In [32]:
# Backpropagation Max-pooling
def max_pooling_backprop(delta, input, pool_size):
    output = np.zeros_like(input)
    for y in range(0, input.shape[0], pool_size[0]):
        for x in range(0, input.shape[1], pool_size[1]):
            patch = input[y:y + pool_size[0], x:x + pool_size[1]]
            max_val = np.max(patch)
            for i in range(pool_size[0]):
                for j in range(pool_size[1]):
                    if patch[i, j] == max_val:
                        output[y + i, x + j] = delta[y // pool_size[0], x // pool_size[1]]
    return output

delta_pooling = max_pooling_backprop(delta_sigmoid, convolved, (2, 2))

print("Max-pooling delta:")
print(delta_pooling)

Max-pooling delta:
[[ 0.          0.          0.06878349  0.        ]
 [ 0.          0.03649574  0.          0.        ]
 [ 0.          0.04684955  0.          0.        ]
 [ 0.          0.          0.         -0.06826229]]


In [39]:
# Convolution backpropagation
def convolution_backprop(image, delta, kernel_shape):
    d_kernel = np.zeros(kernel_shape)
    d_input = np.zeros_like(image)

    for i in range(kernel_shape[0]):
        for j in range(kernel_shape[1]):
            d_kernel[i, j] = np.sum(image[i:image.shape[0] - kernel_shape[0] + i + 1,
                                          j:image.shape[1] - kernel_shape[1] + j + 1] * delta)

    for i in range(delta.shape[0]):
        for j in range(delta.shape[1]):
            d_input[i:i + kernel_shape[0], j:j + kernel_shape[1]] += kernel * delta[i, j]

    return d_kernel, d_input

delta_convolution, delta_input = convolution_backprop(image, delta_pooling, kernel.shape)


print("Convolution delta input:")
print(delta_input)
print()
print("Convolution delta kernel:")
print(delta_convolution)

Convolution delta input:
[[ 0.          0.         -0.00894185  0.01031752  0.        ]
 [ 0.         -0.00474445 -0.02960522  0.04264576  0.        ]
 [ 0.         -0.02470327  0.02965479  0.          0.        ]
 [ 0.         -0.02389327  0.02904672  0.0088741  -0.01023934]
 [ 0.          0.          0.          0.03481377 -0.04232262]]

Convolution delta kernel:
[[0.04474314 0.09433226]
 [0.01082693 0.06850172]]


### Updating

In [43]:
kernel = np.array(kernel)
delta_convolution = np.array(delta_convolution)
fc_weights = np.array(fc_weights)
delta_fc_weights = np.array(delta_fc_weights)
fc_biases = np.array(fc_biases)
delta_fc_biases = np.array(delta_fc_biases)
delta_input = np.array(delta_input)

# Weight updates with learning rate 0.5
eta = 0.5

# Update convolution kernel
kernel_update = kernel - eta * delta_convolution

# Update fully connected layer weights and biases
fc_weights_update = fc_weights - eta * delta_fc_weights
fc_biases_update = fc_biases - eta * delta_fc_biases

print("Updated Convolution Kernel:")
print(kernel_update)
print("\nUpdated Fully Connected Layer Weights:")
print(fc_weights_update)
print("\nUpdated Fully Connected Layer Biases:")
print(fc_biases_update)

Updated Convolution Kernel:
[[-0.15237157  0.10283387]
 [-0.51541346  0.58574914]]

Updated Fully Connected Layer Weights:
[[ 0.53791375  0.74926073  0.87895039 -1.06839493]
 [ 0.06325175 -0.45755644  0.27862977  0.21103696]]

Updated Fully Connected Layer Biases:
[-0.125  0.075]


### Fold and Unfold

In [57]:
import torch
import torch.nn.functional as F


# Convert image and kernel to PyTorch tensors
image_tensor = torch.tensor(image, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
kernel_tensor = torch.tensor(kernel, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

# Convolution using Unfold and Fold
def convolution_unfold(image_tensor, kernel_tensor):
    unfold = F.unfold(image_tensor, kernel_size=(2, 2))
    kernel_flat = kernel_tensor.view(1, -1)
    convolved = kernel_flat @ unfold
    output_height = image_tensor.shape[2] - kernel_tensor.shape[2] + 1
    output_width = image_tensor.shape[3] - kernel_tensor.shape[3] + 1
    convolved = convolved.view(1, 1, output_height, output_width)
    return convolved.squeeze().numpy()

# Perform the convolution
convolved = convolution_unfold(image_tensor, kernel_tensor)
print("Convolved Output:")
print(convolved)

# Max-pooling
def max_pooling(input, pool_size):
    input_tensor = torch.tensor(input, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    pooled = F.max_pool2d(input_tensor, kernel_size=pool_size)
    return pooled.squeeze().numpy()

pooled = max_pooling(convolved, (2, 2))
print("Max-Pooled Output:")
print(pooled)

# Sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

sigmoid_output = sigmoid(pooled)
print("\nOutput after applying sigmoid:")
print(sigmoid_output)

# Fully Connected
fc_weights = np.array([
    [0.61, 0.82, 0.96, -1],
    [0.02, -0.5, 0.23, 0.17]
])
fc_biases = np.array([0, 0])  # Assuming biases are zero for simplicity
flattened = sigmoid_output.flatten()

fc_output = np.dot(flattened, fc_weights.T) + fc_biases
print("\nOutput after fully connected layer:")
print(fc_output)

# Backpropagation
delta_fc = [0.25, -0.15]

# Backpropagation fully connected layer
delta_flattened = np.dot(delta_fc, fc_weights)
delta_fc_weights = np.outer(delta_fc, flattened)
delta_fc_biases = delta_fc

print("fully connected delta input:")
print(delta_flattened)
print()
print("fully connected delta weights:")
print(delta_fc_weights)
print()
print("fully connected delta biases:")
print(delta_fc_biases)

# Backpropagation sigmoid layer
sigmoid_derivative = sigmoid_output * (1 - sigmoid_output)
delta_sigmoid = delta_flattened.reshape(sigmoid_output.shape) * sigmoid_derivative

print("sigmoid delta:")
print(delta_sigmoid)

# Backpropagation Max-pooling
def max_pooling_backprop(delta, input, pool_size):
    output = np.zeros_like(input)
    for y in range(0, input.shape[0], pool_size[0]):
        for x in range(0, input.shape[1], pool_size[1]):
            patch = input[y:y + pool_size[0], x:x + pool_size[1]]
            max_val = np.max(patch)
            for i in range(pool_size[0]):
                for j in range(pool_size[1]):
                    if patch[i, j] == max_val:
                        output[y + i, x + j] = delta[y // pool_size[0], x // pool_size[1]]
    return output

delta_pooling = max_pooling_backprop(delta_sigmoid, convolved, (2, 2))

print("Max-pooling delta:")
print(delta_pooling)

# Convolution backpropagation using Fold and Unfold
def convolution_backprop_unfold(image, delta, kernel_shape):
    image_tensor = torch.tensor(image, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    delta_tensor = torch.tensor(delta, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

    # Use unfold to get patches from the image
    unfold_image = F.unfold(image_tensor, kernel_size=kernel_shape)
    unfold_delta = F.unfold(delta_tensor, kernel_size=(1, 1))

    # Calculate the gradient for the kernel
    d_kernel = torch.matmul(unfold_delta, unfold_image.transpose(1, 2)).view(1, 1, *kernel_shape).squeeze().numpy()

    # Use fold to aggregate gradients back to input space
    delta_expanded = unfold_delta.view(1, -1, delta_tensor.shape[2] * delta_tensor.shape[3])
    d_input_unfold = torch.matmul(kernel_tensor.view(1, -1).t(), delta_expanded)
    d_input = F.fold(d_input_unfold, output_size=image_tensor.shape[2:], kernel_size=kernel_shape)

    return d_kernel, d_input.squeeze().numpy()

delta_convolution, delta_input = convolution_backprop_unfold(image, delta_pooling, kernel.shape)

print("Convolution delta input:")
print(delta_input)
print()
print("Convolution delta kernel:")
print(delta_convolution)

kernel = np.array(kernel)
delta_convolution = np.array(delta_convolution)
fc_weights = np.array(fc_weights)
delta_fc_weights = np.array(delta_fc_weights)
fc_biases = np.array(fc_biases)
delta_fc_biases = np.array(delta_fc_biases)
delta_input = np.array(delta_input)

# Weight updates with learning rate 0.5
eta = 0.5

kernel_update = kernel - eta * delta_convolution
fc_weights_update = fc_weights - eta * delta_fc_weights
fc_biases_update = fc_biases - eta * delta_fc_biases

print("\nUpdated Convolution Kernel:")
print(kernel_update)
print("\nUpdated Fully Connected Layer Weights:")
print(fc_weights_update)
print("\nUpdated Fully Connected Layer Biases:")
print(fc_biases_update)


Convolved Output:
[[ 0.24910003 -0.16479997  0.26520002 -0.34059998]
 [ 0.10890001  0.30920002 -0.2989      0.00980001]
 [-0.22399999  0.612      -0.09120002 -0.2841    ]
 [-0.33089998  0.47700003 -0.2673      0.18919998]]
Max-Pooled Output:
[[0.30920002 0.26520002]
 [0.612      0.18919998]]

Output after applying sigmoid:
[[0.57668996 0.56591415]
 [0.6483969  0.5471594 ]]

Output after fully connected layer:
[ 0.89113214 -0.0292749 ]
fully connected delta input:
[ 0.1495  0.28    0.2055 -0.2755]

fully connected delta weights:
[[ 0.14417249  0.14147854  0.16209923  0.13678984]
 [-0.08650349 -0.08488712 -0.09725954 -0.08207391]]

fully connected delta biases:
[0.25, -0.15]
sigmoid delta:
[[ 0.03649574  0.06878349]
 [ 0.04684955 -0.06826228]]
Max-pooling delta:
[[ 0.          0.          0.06878349  0.        ]
 [ 0.          0.03649574  0.          0.        ]
 [ 0.          0.04684955  0.          0.        ]
 [ 0.          0.          0.         -0.06826229]]
Convolution delta input: